NOTE: This notebook is outdated

# Imports

In [5]:
import sys
sys.path.append('..')
from scripts.extract import convert_xml_to_dataframe
from pathlib import Path
from random import randrange
from datetime import timedelta, datetime, date

import pandas as pd
import random

In [6]:
random.seed(1) # <- Training
# random.seed(2) # <- Validation

# Functions

In [7]:
def random_date(start, end):
    """
    This function will return a random datetime between two datetime
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

# Read data from xml

In [8]:
df = convert_xml_to_dataframe(path='../_shared/data/generated/patients-original.xml')

## Version 2 Generator

In [9]:
# Add boolean for procedures to the dataframe
df['GENDER_MALE'] = [0]
df['GENDER_FEMALE'] = [0]
df['AGE_RANGE_20'] = [0]
df['AGE_RANGE_40'] = [0]
df['AGE_RANGE_60'] = [0]
df['PROCEDURE_A'] = [0]
df['PROCEDURE_B'] = [0]
df['TREATING_PROVIDER_DENTIST'] = [1]
df['TREATING_PROVIDER_FACULTY'] = [0]
df['TREATING_PROVIDER_STUDENT'] = [0]
df['NOICE_MODIFIED'] = [0]
df['STUDENT_ERROR'] = [0]
df = df.drop(['SEX', 'TREATING_PROVIDER'], axis=1)
df = df.drop(0)
df.head()

,PATIENT_ID,BIRTH_DATE,VISIT_DATE,BLEEDING_ON_PROBING,NR_OF_POCKET,NR_OF_FURCATION,NR_OF_MOBILITY,TOTAL_LOSS_OF_ATTACHMENT_LEVEL,HAS_PARODONTITIS,GENDER_MALE,...,AGE_RANGE_20,AGE_RANGE_40,AGE_RANGE_60,PROCEDURE_A,PROCEDURE_B,TREATING_PROVIDER_DENTIST,TREATING_PROVIDER_FACULTY,TREATING_PROVIDER_STUDENT,NOICE_MODIFIED,STUDENT_ERROR


In [10]:
for i in range(100000):
    has_parodontitis = random.randint(0, 2)
    patient_id = 5999977 + i
    treating_provider = random.choice(['dentist', 'faculty', 'student'])

    # Procedures
    procedures_a = 0
    procedures_b = random.randint(0, 1)

    # Birthdate
    d1 = datetime.strptime('1-1-1950', '%d-%m-%Y').date()
    d2 = datetime.strptime('1-1-2002', '%d-%m-%Y').date()
    birth_date = random_date(d1, d2)
    age = calculate_age(birth_date)

    # Gender
    gender = random.choice(['female', 'male'])

    row = {
        'PATIENT_ID': patient_id,
        'GENDER_MALE': int(gender == 'male' if 1 else 0),
        'GENDER_FEMALE': int(gender == 'female' if 1 else 0),
        'BIRTH_DATE': birth_date,
        'AGE_RANGE_20': int(20 <= age < 40 if 1 else 0),
        'AGE_RANGE_40': int(40 <= age < 60 if 1 else 0),
        'AGE_RANGE_60': int(age >= 60 if 1 else 0),
        'VISIT_DATE': '15-01-2008',
        'TREATING_PROVIDER_DENTIST': int(treating_provider == 'dentist' if 1 else 0),
        'TREATING_PROVIDER_FACULTY': int(treating_provider == 'faculty' if 1 else 0),
        'TREATING_PROVIDER_STUDENT': int(treating_provider == 'student' if 1 else 0),
        'PROCEDURE_A': procedures_a,
        'PROCEDURE_B': procedures_b,
        'BLEEDING_ON_PROBING': random.randint(0, 25),       # percentage of bleeding (0-100)
        'NR_OF_POCKET': random.randint(0, 1),               # scale 0-4
        'NR_OF_FURCATION': 0,                               # >0 parodontitis, scale 0-5
        'NR_OF_MOBILITY': random.randint(0, 1),
        'TOTAL_LOSS_OF_ATTACHMENT_LEVEL': 0,
        'HAS_PARODONTITIS': 0,
        # For debug purposes only
        'NOICE_MODIFIED': 0,
        'STUDENT_ERROR': 0,
    }

    if has_parodontitis == 1 or (random.randint(0, 100) <= 40 and row['AGE_RANGE_40'] == 1) or (random.randint(0, 100) <= 60 and row['AGE_RANGE_60'] == 1):
        row['BLEEDING_ON_PROBING'] = random.randint(20, 100)
        row['NR_OF_POCKET'] = random.randint(1, 4)
        row['NR_OF_FURCATION'] = random.randint(1, 5)
        row['NR_OF_MOBILITY'] = random.randint(2, 3)
        row['TOTAL_LOSS_OF_ATTACHMENT_LEVEL'] = random.randint(1, 3)
        row['HAS_PARODONTITIS'] = 1

        # Someone with parodontitis has a 70% chance to have had procedure A
        if random.randint(0, 100) <= 70:
            row['PROCEDURE_A'] = 1

        # Add some noice to the data
        if random.randint(0, 100) <= 2:
            row['HAS_PARODONTITIS'] = 0
            row['NOICE_MODIFIED'] = 1

        # A student has a 50% chance to make an error
        if random.randint(0, 100) <= 50 and row['TREATING_PROVIDER_STUDENT'] == 1:
            row['HAS_PARODONTITIS'] = 0
            row['STUDENT_ERROR'] = 1

    new_record = pd.DataFrame(row, index=[0])
    df = pd.concat([df, new_record], ignore_index=True)

## Check amount of total people having parodontitis

In [11]:
amount_of_people_without_parodontitis = df[df["HAS_PARODONTITIS"] == 0]
amount_of_people_with_parodontitis = df[df["HAS_PARODONTITIS"] == 1]
print(f'People without parodontitis: {len(amount_of_people_without_parodontitis)}')
print(f'People with parodontitis: {len(amount_of_people_with_parodontitis)}')

People without parodontitis: 56592
People with parodontitis: 43408


In [12]:
filepath = Path('../_shared/data/generated/patients-v6.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(filepath, index=False)